# Read DWD CDC Time Series, Merge with Station Description and Append 

The main idea behind this activity is to reformat and merge time series (here we use hourly precipitation) from the DWD Climate Data Center in such a way that it can be used with the **QGIS time manager extension**. 

This extension allows to filter an attribute table of a vector layer (e.g. points representing precipitation stations plus precipitation data) with a time stamp column. The extension limits the attribute table to the records matching the particular time stamp provided by the time manager extension (e.g. by the user moving the time slider). This selected subset of the attribute table is then used to change the sympology of the vector layer according to the variable of interest (e.g. precipitation rate).

The QGIS time manager extension approach is a bit brute force, because each individual measurement at a station at a given time is one feature (row in the table), i.e. a time series at station X with hourly resolution for a day (24 values) entails 24 different features with the same station id and the corresponding coordinates but different times. As of now this 1:n relationship can only be realized by importing a CSV file with the according structure. 

(At least I was not able to generate the required view on a 1:n relationship by merging a point vector layer with precipitation station locations and an imported CSV time series table.)

The final data format is a concatenation of time series together with geographic location in 2D (e.g. lat, lon). The required data format looks principly like this:


| station_id |        name        |   lat   |   lon  |        meas_time       | prec_rate |
|:----------:|:------------------:|:-------:|:------:|:----------------------:|:---------:|
|        ... | ...                |     ... |    ... |                    ... |       ... |
|       1595 | Gelsenkirchen-Buer | 51.5762 | 7.0652 | 2018-12-07T08:00:00UTC |       1.5 |
|       1595 | Gelsenkirchen-Buer | 51.5762 | 7.0652 | 2018-12-07T09:00:00UTC |       1.7 |
|       1595 | Gelsenkirchen-Buer | 51.5762 | 7.0652 | 2018-12-07T10:00:00UTC |       0.1 |
|        ... | ...                |     ... |    ... |                    ... |       ... |
|      13670 | Duisburg-Baerl     | 51.5088 | 6.7018 | 2018-12-07T08:00:00UTC |       0.8 |
|      13670 | Duisburg-Baerl     | 51.5088 | 6.7018 | 2018-12-07T09:00:00UTC |       0.4 |
|      13670 | Duisburg-Baerl     | 51.5088 | 6.7018 | 2018-12-07T10:00:00UTC |       0.0 |
|        ... | ...                |     ... |    ... |                    ... |       ... |


(Table generated with https://www.tablesgenerator.com/markdown_tables)

To achieve this the precipitation time series (station_id, meas_time, prec_rate) have to be merged with the station metadata (station_id, lat, lon) coming from the a CSV file generated in an earlier activity. We use Pandas to read, join and append the data to generate the final CSV file to be imported as a point layer to QGIS. 

This final data format is far from being optimal because of large size and highly redundant information. This is a challenge for QGIS which loses responsiveness with large data. To jsut show the principle it is advisable to limit to size of the problem. 

The following filters (selection criteria) are applied:

  * Precipitation stations in NRW only (approx. 127 stations) 
  * Hourly precipitation data
  * Time interval from 2018-12-01 to last date in precipitation data set 
  
Still: 40 days * 24 hrs / day * 127 stations = 121920 records leading to 121920 features in a point layer in QGIS. 

In fact, the resulting number of records is arround 91000. The reason might be that not all stations in the station list have time series. This has to be checked carefully.

## FTP Connection

* FTP: ftp://opendata.dwd.de/climate_environment/CDC/observations_germany/
* HTTPS: https://opendata.dwd.de/climate_environment/CDC/observations_germany/

### Connection Parameters

In [1]:
server = "opendata.dwd.de"
user   = "anonymous"
passwd = ""

### FTP Directory Definition and Station Description Filename Pattern

In [2]:
# The topic of interest.
topic_dir = "/hourly/precipitation/recent/"
#topic_dir = "/annual/kl/historical/"

# This is the search pattern common to ALL station description file names 
station_desc_pattern = "_Beschreibung_Stationen.txt"

# Below this directory tree node all climate data are stored.
ftp_climate_data_dir = "/climate_environment/CDC/observations_germany/climate/"
ftp_dir =  ftp_climate_data_dir + topic_dir

### Local Directories

In [3]:
local_ftp_dir         = "../data/original/DWD/"      # Local directory to store local ftp data copies, the local data source or input data. 
local_ftp_station_dir = local_ftp_dir + topic_dir # Local directory where local station info is located
local_ftp_ts_dir      = local_ftp_dir + topic_dir # Local directory where time series downloaded from ftp are located

local_generated_dir   = "../data/generated/DWD/" # The generated of derived data in contrast to local_ftp_dir
local_station_dir     = local_generated_dir + topic_dir # Derived station data, i.e. the CSV file
local_ts_merged_dir   = local_generated_dir + topic_dir # Parallelly merged time series, wide data frame with one TS per column
local_ts_appended_dir = local_generated_dir + topic_dir # Serially appended time series, long data frame for QGIS TimeManager Plugin


In [4]:
print(local_ftp_dir)
print(local_ftp_station_dir)
print(local_ftp_ts_dir)
print()
print(local_generated_dir)
print(local_station_dir)
print(local_ts_merged_dir)
print(local_ts_appended_dir)

../data/original/DWD/
../data/original/DWD//hourly/precipitation/recent/
../data/original/DWD//hourly/precipitation/recent/

../data/generated/DWD/
../data/generated/DWD//hourly/precipitation/recent/
../data/generated/DWD//hourly/precipitation/recent/
../data/generated/DWD//hourly/precipitation/recent/


In [5]:
import os
os.makedirs(local_ftp_dir,exist_ok = True) # it does not complain if the dir already exists.
os.makedirs(local_ftp_station_dir,exist_ok = True)
os.makedirs(local_ftp_ts_dir,exist_ok = True)

os.makedirs(local_generated_dir,exist_ok = True)
os.makedirs(local_station_dir,exist_ok = True)
os.makedirs(local_ts_merged_dir,exist_ok = True)
os.makedirs(local_ts_appended_dir,exist_ok = True)

### FTP Connect

In [6]:
import ftplib
ftp = ftplib.FTP(server)
res = ftp.login(user=user, passwd = passwd)
print(res)

230 Login successful.


In [7]:
ret = ftp.cwd(".")

In [8]:
#ftp.quit()

### Generate Pandas Dataframe from FTP Directory Listing

In [9]:
from my_dwd import gen_df_from_ftp_dir_listing

ModuleNotFoundError: No module named 'my_dwd'

In [ ]:
df_ftpdir = gen_df_from_ftp_dir_listing(ftp, ftp_dir)

In [10]:
df_ftpdir.head(5)

NameError: name 'df_ftpdir' is not defined

### Dataframe with TS Zip Files

In [101]:
#df_ftpdir["ext"]==".zip"
df_zips = df_ftpdir[df_ftpdir["ext"]==".zip"]
df_zips.set_index("station_id", inplace = True)
df_zips.head(5)

,name,ext,size,type
station_id,,,,
20,stundenwerte_RR_00020_akt.zip,.zip,43912,-
44,stundenwerte_RR_00044_akt.zip,.zip,44209,-
53,stundenwerte_RR_00053_akt.zip,.zip,42391,-
71,stundenwerte_RR_00071_akt.zip,.zip,15127,-
73,stundenwerte_RR_00073_akt.zip,.zip,43403,-


### Download the Station Description File

In [102]:
import pandas as pd

In [103]:
from my_dwd import grabFile

In [104]:
station_fname = df_ftpdir[df_ftpdir['name'].str.contains(station_desc_pattern)]["name"].values[0]
print("Station description file name:\n%s" % (station_fname))

# ALternative
#station_fname2 = df_ftpdir[df_ftpdir["name"].str.match("^.*Beschreibung_Stationen.*txt$")]["name"].values[0]
#print(station_fname2)

Station description file name:
RR_Stundenwerte_Beschreibung_Stationen.txt


In [105]:
src = ftp_dir + station_fname
dest = local_ftp_station_dir + station_fname
print("grabFile(ftp, src, dest):")
print("FTP source: " + src)
print("Local dest:   " + dest)
grabFile(ftp, src, dest)

grabFile(ftp, src, dest):
FTP source: /climate_environment/CDC/observations_germany/climate//hourly/precipitation/recent/RR_Stundenwerte_Beschreibung_Stationen.txt
Local dest:   ../data/original/DWD//hourly/precipitation/recent/RR_Stundenwerte_Beschreibung_Stationen.txt


In [106]:
# extract column names. They are in German (de)
# We have to use codecs because of difficulties with character encoding (German Umlaute)
import codecs

def station_desc_txt_to_csv(txtfile, csvfile):
    file = codecs.open(txtfile,"r","utf-8")
    r = file.readline()
    file.close()
    colnames_de = r.split()
    colnames_de
    
    translate = \
    {'Stations_id':'station_id',
     'von_datum':'date_from',
     'bis_datum':'date_to',
     'Stationshoehe':'altitude',
     'geoBreite': 'latitude',
     'geoLaenge': 'longitude',
     'Stationsname':'name',
     'Bundesland':'state'}
    
    colnames_en = [translate[h] for h in colnames_de]
    
    # Skip the first two rows and set the column names.
    df = pd.read_fwf(txtfile,skiprows=2,names=colnames_en, parse_dates=["date_from","date_to"],index_col = 0)
    
    # write csv
    df.to_csv(csvfile, sep = ";")
    return(df)

In [107]:
basename = os.path.splitext(station_fname)[0]
df_stations = station_desc_txt_to_csv(local_ftp_station_dir + station_fname, local_station_dir + basename + ".csv")
df_stations.head()

,date_from,date_to,altitude,latitude,longitude,name,state
station_id,,,,,,,
3,1995-09-01,2011-04-01,202,50.7827,6.0941,Aachen,Nordrhein-Westfalen
20,2004-08-14,2021-02-01,432,48.9220,9.9129,Abtsgmünd-Untergröningen,Baden-Württemberg
44,2007-04-01,2021-02-01,44,52.9336,8.2370,Großenkneten,Niedersachsen
53,2005-10-01,2021-02-01,60,52.5850,13.5634,Ahrensfelde,Brandenburg
71,2004-10-22,2020-01-01,759,48.2156,8.9784,Albstadt-Badkap,Baden-Württemberg


### Select Stations Located in NRW and Operational 

In [108]:
#station_ids_selected = df_stations[df_stations['state'].str.contains("Nordrhein")].index
#station_ids_selected

In [109]:
# Create variable with TRUE if state is Nordrhein-Westfalen

# isNRW = df_stations['state'] == "Nordrhein-Westfalen"
isNRW = df_stations['state'].str.contains("Nordrhein")

# Create variable with TRUE if date_to is latest date (indicates operation up to now)
isOperational = df_stations['date_to'] == df_stations.date_to.max() 

#isBefore1950 = df_stations['date_from'] < '1950'
#dfNRW = df_stations[isNRW & isOperational & isBefore1950]

# select on both conditions

dfNRW = df_stations[isNRW & isOperational]

#print("Number of stations in NRW: \n", dfNRW.count())
dfNRW

,date_from,date_to,altitude,latitude,longitude,name,state
station_id,,,,,,,
216,2004-10-01,2021-02-01,298,51.1143,7.8807,Attendorn-Neulisternohl,Nordrhein-Westfalen
389,2009-11-01,2021-02-01,436,51.0148,8.4318,"Berleburg, Bad-Arfeld",Nordrhein-Westfalen
554,1995-09-01,2021-02-01,23,51.8293,6.5365,Bocholt-Liedern (Wasserwerk),Nordrhein-Westfalen
603,1999-03-03,2021-02-01,147,50.7293,7.2040,Königswinter-Heiderhof,Nordrhein-Westfalen
613,2004-11-01,2021-02-01,206,51.5677,9.2324,Borgentreich,Nordrhein-Westfalen
...,...,...,...,...,...,...,...
13671,2007-12-01,2021-02-01,221,50.9655,7.2753,Overath-Böke,Nordrhein-Westfalen
13696,2007-12-01,2021-02-01,60,51.5966,7.4048,Waltrop-Abdinghof,Nordrhein-Westfalen
13700,2008-05-01,2021-02-01,205,51.3329,7.3411,Gevelsberg-Oberbröking,Nordrhein-Westfalen


### Download TS Data from FTP Server

Problem: Not all stations listed in the station description file are associated with a time series (zip file)! The stations in the description file and the set of stations whoch are TS data provided for (zip files) do not match perfectly.  

In [110]:
#list(dfNRW.index)

In [111]:
# Add the names of the zip files only to a list. 
local_zip_list = []

station_ids_selected = list(dfNRW.index)

for station_id in station_ids_selected:
    try:
        fname = df_zips["name"][station_id]
        print(fname)
        grabFile(ftp, ftp_dir + fname, local_ftp_ts_dir + fname)
        local_zip_list.append(fname)
    except:
        print("WARNING: TS file for key %d not found in FTP directory." % station_id)

stundenwerte_RR_00216_akt.zip
stundenwerte_RR_00389_akt.zip
stundenwerte_RR_00554_akt.zip
stundenwerte_RR_00603_akt.zip
stundenwerte_RR_00613_akt.zip
stundenwerte_RR_00617_akt.zip
stundenwerte_RR_00644_akt.zip
stundenwerte_RR_00796_akt.zip
stundenwerte_RR_00871_akt.zip
stundenwerte_RR_00902_akt.zip
stundenwerte_RR_00934_akt.zip
stundenwerte_RR_00989_akt.zip
stundenwerte_RR_01024_akt.zip
stundenwerte_RR_01046_akt.zip
stundenwerte_RR_01078_akt.zip
stundenwerte_RR_01241_akt.zip
stundenwerte_RR_01246_akt.zip
stundenwerte_RR_01300_akt.zip
stundenwerte_RR_01303_akt.zip
stundenwerte_RR_01327_akt.zip
stundenwerte_RR_01590_akt.zip
stundenwerte_RR_01595_akt.zip
stundenwerte_RR_01766_akt.zip
stundenwerte_RR_02027_akt.zip
stundenwerte_RR_02110_akt.zip
stundenwerte_RR_02254_akt.zip
stundenwerte_RR_02473_akt.zip
stundenwerte_RR_02483_akt.zip
stundenwerte_RR_02497_akt.zip
stundenwerte_RR_02629_akt.zip
stundenwerte_RR_02667_akt.zip
stundenwerte_RR_02810_akt.zip
stundenwerte_RR_02947_akt.zip
stundenwer

In [112]:
#local_zip_list

### Join (Merge) the Time Series Columns

https://medium.com/@chaimgluck1/working-with-pandas-fixing-messy-column-names-42a54a6659cd


In [25]:
from zipfile import ZipFile

In [26]:
# PRECIPITATION
def prec_ts_merge():
    # Very compact code.
    df = pd.DataFrame()
    for elt in local_zip_list:
        ffname = local_ftp_ts_dir + elt
        print("Zip archive: " + ffname)
        with ZipFile(ffname) as myzip:
            # read the time series data from the file starting with "produkt"
            prodfilename = [elt for elt in myzip.namelist() if elt.split("_")[0]=="produkt"][0] 
            print("Extract product file: %s" % prodfilename)
            print()
            with myzip.open(prodfilename) as myfile:
                dftmp = prec_ts_to_df(myfile)
                s = dftmp["r1"].rename(dftmp["stations_id"][0]).to_frame()
                # outer merge.
                df = pd.merge(df, s, left_index=True, right_index=True, how='outer')

    #df.index.names = ["year"]
    df.index.rename(name = "time", inplace = True)
    return(df)

In [27]:
# TEMPERATURE
def temp_ts_merge():
    # Very compact code.
    df = pd.DataFrame()
    for elt in local_zip_list:
        ffname = local_ftp_ts_dir + elt
        print("Zip archive: " + ffname)
        with ZipFile(ffname) as myzip:
            # read the time series data from the file starting with "produkt"
            prodfilename = [elt for elt in myzip.namelist() if elt.split("_")[0]=="produkt"][0] 
            print("Extract product file: %s" % prodfilename)
            print()
            with myzip.open(prodfilename) as myfile:
                dftmp = temp_ts_to_df(myfile)
                s = dftmp["ja_tt"].rename(dftmp["stations_id"][0]).to_frame()
                # outer merge.
                df = pd.merge(df, s, left_index=True, right_index=True, how='outer')

    #df.index.names = ["year"]
    df.index.rename(name = "time", inplace = True)
    return(df)

In [28]:
df_merged_ts = prec_ts_merge()

Zip archive: ../data/original/DWD//hourly/precipitation/recent/stundenwerte_RR_00216_akt.zip
Extract product file: produkt_rr_stunde_20190801_20210131_00216.txt

Zip archive: ../data/original/DWD//hourly/precipitation/recent/stundenwerte_RR_00389_akt.zip
Extract product file: produkt_rr_stunde_20190801_20210131_00389.txt

Zip archive: ../data/original/DWD//hourly/precipitation/recent/stundenwerte_RR_00554_akt.zip
Extract product file: produkt_rr_stunde_20190801_20210131_00554.txt

Zip archive: ../data/original/DWD//hourly/precipitation/recent/stundenwerte_RR_00603_akt.zip
Extract product file: produkt_rr_stunde_20190801_20210131_00603.txt

Zip archive: ../data/original/DWD//hourly/precipitation/recent/stundenwerte_RR_00613_akt.zip
Extract product file: produkt_rr_stunde_20190801_20210131_00613.txt

Zip archive: ../data/original/DWD//hourly/precipitation/recent/stundenwerte_RR_00617_akt.zip
Extract product file: produkt_rr_stunde_20190801_20210131_00617.txt

Zip archive: ../data/origina

In [32]:
df_merged_ts

,216,389,554,603,613,617,644,796,871,902,...,7344,7374,7378,13669,13670,13671,13696,13700,13713,15000
time,,,,,,,,,,,,,,,,,,,,,
2019-08-01 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-08-01 01:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-08-01 02:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-08-01 03:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-08-01 04:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-01-31 19:00:00,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021-01-31 20:00:00,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1
2021-01-31 21:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7


In [85]:
import pandas as pd
import pytz

index = pd.date_range('20140101 21:55', freq='15S', periods=5)
df = pd.DataFrame(1, index=index, columns=['X'])
print(df)
#                      X
# 2014-01-01 21:55:00  1
# 2014-01-01 21:55:15  1
# 2014-01-01 21:55:30  1
# 2014-01-01 21:55:45  1
# 2014-01-01 21:56:00  1

# [5 rows x 1 columns]
print(df.index)
# <class 'pandas.tseries.index.DatetimeIndex'>
# [2014-01-01 21:55:00, ..., 2014-01-01 21:56:00]
# Length: 5, Freq: 15S, Timezone: None

eastern = pytz.timezone('US/Eastern')
#rb: df.index = df.index.tz_localize(pytz.utc).tz_convert(eastern)
df.index = df.index.tz_localize(pytz.utc)
print(df)
#                            X
# 2014-01-01 16:55:00-05:00  1
# 2014-01-01 16:55:15-05:00  1
# 2014-01-01 16:55:30-05:00  1
# 2014-01-01 16:55:45-05:00  1
# 2014-01-01 16:56:00-05:00  1

# [5 rows x 1 columns]

print(df.index)
# <class 'pandas.tseries.index.DatetimeIndex'>
# [2014-01-01 16:55:00-05:00, ..., 2014-01-01 16:56:00-05:00]
# Length: 5, Freq: 15S, Timezone: US/Eastern

                     X
2014-01-01 21:55:00  1
2014-01-01 21:55:15  1
2014-01-01 21:55:30  1
2014-01-01 21:55:45  1
2014-01-01 21:56:00  1
DatetimeIndex(['2014-01-01 21:55:00', '2014-01-01 21:55:15',
               '2014-01-01 21:55:30', '2014-01-01 21:55:45',
               '2014-01-01 21:56:00'],
              dtype='datetime64[ns]', freq='15S')
                           X
2014-01-01 21:55:00+00:00  1
2014-01-01 21:55:15+00:00  1
2014-01-01 21:55:30+00:00  1
2014-01-01 21:55:45+00:00  1
2014-01-01 21:56:00+00:00  1
DatetimeIndex(['2014-01-01 21:55:00+00:00', '2014-01-01 21:55:15+00:00',
               '2014-01-01 21:55:30+00:00', '2014-01-01 21:55:45+00:00',
               '2014-01-01 21:56:00+00:00'],
              dtype='datetime64[ns, UTC]', freq='15S')


In [113]:

csvfname = "prec_ts_appended_3_cols.csv"

first = True

#for elt in local_zip_list[0:1]:
for elt in local_zip_list:
    ffname = local_ftp_ts_dir + elt
    print("Zip archive: " + ffname)
    with ZipFile(ffname) as myzip:
        # read the time series data from the file starting with "produkt"
        prodfilename = [elt for elt in myzip.namelist() if elt.split("_")[0]=="produkt"][0] 
        print("Extract product file: %s" % prodfilename)
        print()
        with myzip.open(prodfilename) as myfile:
            dftmp = prec_ts_to_df(myfile)[["stations_id","r1"]]
            # dftmp.to_csv(f, header=f.tell()==0)
            if (first):
                first = False
                dftmp.to_csv(csvfname, mode = "w", header = True)
            else:
                dftmp.to_csv(csvfname, mode = "a", header = False)
                

Zip archive: ../data/original/DWD//hourly/precipitation/recent/stundenwerte_RR_00216_akt.zip
Extract product file: produkt_rr_stunde_20190801_20210131_00216.txt

Zip archive: ../data/original/DWD//hourly/precipitation/recent/stundenwerte_RR_00389_akt.zip
Extract product file: produkt_rr_stunde_20190801_20210131_00389.txt

Zip archive: ../data/original/DWD//hourly/precipitation/recent/stundenwerte_RR_00554_akt.zip
Extract product file: produkt_rr_stunde_20190801_20210131_00554.txt

Zip archive: ../data/original/DWD//hourly/precipitation/recent/stundenwerte_RR_00603_akt.zip
Extract product file: produkt_rr_stunde_20190801_20210131_00603.txt

Zip archive: ../data/original/DWD//hourly/precipitation/recent/stundenwerte_RR_00613_akt.zip
Extract product file: produkt_rr_stunde_20190801_20210131_00613.txt

Zip archive: ../data/original/DWD//hourly/precipitation/recent/stundenwerte_RR_00617_akt.zip
Extract product file: produkt_rr_stunde_20190801_20210131_00617.txt

Zip archive: ../data/origina

In [114]:
df = pd.read_csv(csvfname, index_col = "mess_datum")

In [115]:
df.head()

,stations_id,r1
mess_datum,,
2019-08-01 00:00:00,216,0.0
2019-08-01 01:00:00,216,0.0
2019-08-01 02:00:00,216,0.0
2019-08-01 03:00:00,216,0.0
2019-08-01 04:00:00,216,0.0


In [56]:
from datetime import datetime

In [78]:
idx = (df.index < '2021-01-01') & (df.index >= '2020-01-01') & (df["stations_id"] == 216)
df2 = df[idx]
df2.to_csv("bla.csv")

In [81]:
df2[(df2.index >= "2020-03-29") & (df2.index < "2020-03-30")]

,stations_id,r1
mess_datum,,
2020-03-29 00:00:00,216,0.0
2020-03-29 01:00:00,216,0.0
2020-03-29 02:00:00,216,0.0
2020-03-29 03:00:00,216,0.0
2020-03-29 04:00:00,216,0.0
2020-03-29 05:00:00,216,0.0
2020-03-29 06:00:00,216,0.0
2020-03-29 07:00:00,216,0.0
2020-03-29 08:00:00,216,0.0
